# Dirichlet Distribution

*A multinomial generalization of Beta distribution*


$f(x_1,\dots,x_k;\alpha_1,\dots,\alpha_K) = \dfrac{1}{B(\alpha)}\displaystyle \prod_{i=1}^{K} x_i^{\alpha_i-1}$<br><br>

$B(\alpha) = \dfrac{\displaystyle \prod_{i=1}^{K}\Gamma(\alpha_i)}{\Gamma(\displaystyle \sum_{i=1}^{K}\alpha_i)}$

# Dirichlet Process

Dirichlet processes (after Peter Gustav Lejeune Dirichlet) are a family of stochastic processes whose realizations are probability distributions

Dirichlet Process is a distribution over distribution parameterized by $\alpha$ (dispersion parameter) and $G$(Base Distribution)

Dirichlet Process can be demostrated by three models:<br>
1.*Chinese Restaurant Process*<br>
2.*Polya Urn Process*<br>
3.*Stick Breaking Process*<br>

Algorithm for simulation of generation of $X_1,X_2,\dots,$ by Dirichlet Process:

1.Draw $X_1$ from the base distribution $G$<br>
2.For $n>1$:<br>

-With the probability $\dfrac{\alpha}{n+\alpha}$ sample $X_n$ from $G$

-With the probability $\dfrac{n_x}{n+\alpha}$ set $X_x=x,$  n_x := |${$:X_j=x$ and $j<n$| where $|.|$ denotes the number of elements in the set

# Properties of DP

Let DP($\alpha$, $G$ ) be the Dirichlet Process with dispersion parameter $\alpha$ and base distribution $G$, then<br><br>

<center>$\mathbf{E}_{DP(\alpha,G)}[x] = \mathbf{E}_G[x]$<br><br>
As $\alpha \rightarrow \infty,  DP(\alpha,G) \rightarrow G$</center>


# Gibbs Sampling for DPGMM

*Base distribution is Gaussain distribution*
<br>
<center>$p(z_i = k | z_{-i}) \equiv p(z_i | z_1 \dots z_{i-1},z_{i+1},\dots z_m)$</center>

<center>$p(z_i = k|z_{-i},\vec{x},{\theta_k},\alpha) $<br><br>
$= p(z_i = k|z_{-i},x_i,\vec{x},\theta_k,\alpha) $<br><br>
$= p(z_i = k|z_{-i},\alpha)p(x_i|\theta_k,\vec{x})$</center>

\begin{cases}
({\dfrac{n_k}{n+\alpha}})\mathcal{N}(x,\dfrac{n_x}{n+1},\mathbf{1}),\space existing\\
({\dfrac{\alpha}{n+\alpha}})\mathcal{N}(x,0,1)),\space new
\end{cases}

where,<br>
$z_i$ is the group assignment of $x_i$<br>
$k$ is the cluster label<br>
$\alpha$ is the dispersion parameter

In [1]:
#nbi:hide_in
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact
from scipy.stats import dirichlet, multivariate_normal
from scipy import stats
import math
import pandas as pd
from sklearn import datasets
from sklearn.cluster import KMeans

In [2]:
#nbi:hide_in
def Dir_Dist(a1, a2, a3):
    alpha = np.array([a1,a2,a3])
    theta = stats.dirichlet(alpha).rvs(1000)
    fig = plt.figure(figsize=(8, 8), dpi=100)
    ax = plt.gca(projection='3d')
    plt.title(r'$\alpha$ = {}'.format(alpha))
    ax.scatter(theta[:, 0], theta[:, 1], theta[:, 2])
    ax.view_init(azim=30)
    ax.set_xlabel(r'$\theta_1$')
    ax.set_ylabel(r'$\theta_2$')
    ax.set_zlabel(r'$\theta_3$')
    plt.show()

In [3]:
#nbi:hide_in
interact(Dir_Dist, a1=(0.1,2),a2=(0.1,2),a3=(0.1,2))

interactive(children=(FloatSlider(value=1.05, description='a1', max=2.0, min=0.1), FloatSlider(value=1.05, des…

<function __main__.Dir_Dist(a1, a2, a3)>

In [4]:
#nbi:hide_in
def dirichlet_process(h_0, alpha):
    """
    Truncated dirichlet process.
    :param h_0: (scipy distribution)
    :param alpha: (flt)
    :param n: (int) Truncate value.
    """
    n = max(int(5 * alpha + 2), 500)  # truncate the values. 
    pi = stats.beta(1, alpha).rvs(size=n)
    pi[1:] = pi[1:] * (1 - pi[:-1]).cumprod()  # stick breaking process
    theta = h_0(size=n)  # samples from original distribution
    return pi, theta
        
def plot_normal_dp_approximation(alpha, n=3):
    pi, theta = dirichlet_process(stats.norm.rvs, alpha)
    x = np.linspace(-4, 4, 100)
    
    plt.figure(figsize=(14, 4))
    plt.suptitle(r'Three samples from DP($\alpha$). $\alpha$ = {}'.format(alpha))
    plt.ylabel(r'$\pi$')
    plt.xlabel(r'$\theta$')
    pltcount = int('1' + str(n) + '0')
    
    for i in range(n):
        pltcount += 1
        plt.subplot(pltcount)
        pi, theta = dirichlet_process(stats.norm.rvs, alpha)
        pi = pi * (stats.norm.pdf(0) / pi.max())
        plt.vlines(theta, 0, pi, alpha=0.5)
        plt.ylim(0, 1)
        plt.plot(x, stats.norm.pdf(x))
        #print(len(theta),len(pi))
    plt.show()...

In [5]:
#nbi:hide_in
interact(plot_normal_dp_approximation, alpha=(1, 50), n=(2,3))

interactive(children=(IntSlider(value=25, description='alpha', max=50, min=1), IntSlider(value=3, description=…

<function __main__.plot_normal_dp_approximation(alpha, n=3)>

In [6]:
#nbi:hide_in
def chinese_restaurant_process_clustering(alpha,seed,k_,num_cluster):
    #print("\nComputing KMeans with KMeans++ Initialisation....\n")
    km = KMeans(n_clusters=k_,max_iter=20)
    #print("Computed KMeans with initialisation!\n")
    """
        Here we are adding the dataset - since we are going to be performing gaussian
        mixture model clustering using dirichlet processes, we are going to sample the
        data points from a mixture of gaussians - just for this demo, we are setting cluster 
        sizes arbitrarily"""
    np.random.seed(seed)
    
    dataset_1 = np.random.multivariate_normal([5, 5], np.diag([0.5, 0.5]), size=10)
    dataset_2 = np.random.multivariate_normal([7.5, 8], np.diag([0.5, 0.5]), size=60)
    dataset_3 = np.random.multivariate_normal([10, 12], np.diag([0.5, 0.5]), size=40)
    dataset_4 = np.random.multivariate_normal([6,11],np.diag([1,1]),size=50)
    dataset_5 = np.random.multivariate_normal([9,11],np.diag([1,1]),size=60)
    dataset_6 = np.random.multivariate_normal([10,6],np.diag([0.5,0.5]),size=50)
    
    dataset_total = np.vstack([dataset_1,dataset_2,dataset_3,dataset_4,dataset_5,dataset_6])
    
    if num_cluster<=6:
        arr = [10,70,110,160,220,270]
        dataset_total = dataset_total[:arr[num_cluster-1]]
    else:
        temp = num_cluster-6
        new_cluster = np.random.multivariate_normal([10,9],np.diag([0.75, 0.75]), size=50)
        dataset_total = np.vstack([dataset_total,new_cluster])
        new_cluster = np.random.multivariate_normal([0,6],np.diag([0.75, 0.75]), size=50)
        dataset_total = np.vstack([dataset_total,new_cluster])
        
    """
    Just combine all the data points that were generated into a single vector
    This is useful since we can later use these as binary indices to find out the 
    respective clusters"""

    N, D = dataset_total.shape
    """
        Note that N gives the number of samples in the dataset
        D is the number of dimensions. Only 2D Case is visualisable
        other n-D we can just use the data accordingly"""

    """
        Beginning of the Gaussian Mixture Model 
        We are going to be assuming that all points belong to the same cluster
        initially. Then we shall, decompose the data into clusters or combine them
        to have reduced clusters"""
    multi_variate_normal = multivariate_normal # We are using the Higher Dimensional Gaussian
    means = []  # This is the list that contains the means of the Gaussians that we fit for each cluster
    sigma = np.eye(D) # This is the covariance matrix - for simplicity's sake we are setting the covariance matrix
    prec = np.linalg.inv(sigma)  # Fixed precision matrix for all Gaussians
    zs = np.zeros([N], dtype=int)  # Latent variables that need to be inferred
    C = []  # This is a list of arrays, each of size number of datapoints. The arrays are binary and indicate
            # whether a particular element belongs to a cluster or not. Note that each element belongs to one cluster,
            # the cluster with its highest probability
    samples_per_cluster = []  # Count of each cluster
    start_mean = np.ones(D)
    start_variance = np.eye(D)
    prec0 = np.linalg.inv(np.eye(D))
    G = multi_variate_normal(mean=start_mean, cov=start_variance)## Base Distribution
    C.append(np.ones(N, dtype=int))
    zs[:] = 0        ## Sets all the elements to 0 - assume that all points belonged to the same cluster
    samples_per_cluster.append(N)   #Initially all points in same cluster
    means.append(G.rvs())           # Add the mean of the starting cluster
    K = 1                           # Number of Clusters is now, 1.
    #print("Gibbs Sampling Started....\n")
    for iteration in range(20):     # Heuristic - Can be anything of the users choice.
        """Gibbs Sampling starts here
           Note that we are simulating the Chinese Restaurant Process for our model
           to cluster the data points"""
        
        for i in range(N):          # Gibbs Sampling Begin
            """
                Unassign this particular point.
                In each iteration, assign each point to a new 
                cluster with probability 1/(alpha+n) or add to an existing
                cluster with probability proportional to the number of points in every
                cluster, i.e., (n_k)/(n+alpha), where n_k is the number of points in cluster k
                """
            
            zi = zs[i]              
            C[zi][i] = 0
            samples_per_cluster[zi] -= 1
            """
                Suppose after unassigning, number of clusters goes to zero.
                """
            if samples_per_cluster[zi] == 0:
                zs[zs > zi] -= 1
                del C[zi]
                del samples_per_cluster[zi]
                del means[zi]
                K -= 1
            probs = np.zeros(K+1)
            zs_minus_i = zs[np.arange(len(zs)) != i]
            """
                This is the step where Gibbs Sampling Starts
                We normalize the probability since we need to multiply the likelihood 
                as per max-likelihood principle
            """
            for k in range(K):
                nk_minus = zs_minus_i[zs_minus_i == k].shape[0]
                crp = nk_minus / (N + alpha - 1)
                probs[k] = crp * multi_variate_normal.pdf(dataset_total[i], means[k], sigma)
            crp = alpha / (N + alpha - 1)
            likelihood = multi_variate_normal.pdf(dataset_total[i], start_mean, start_variance+sigma)
            probs[K] = crp*likelihood
            probs /= np.sum(probs)
            z = np.random.multinomial(n=1, pvals=probs).argmax()
            if z == K:
                C.append(np.zeros(N, dtype=int))
                samples_per_cluster.append(0)
                means.append(G.rvs())
                K+=1
            zs[i] = z
            C[z][i] = 1
            samples_per_cluster[z] += 1
        """
            Sampling is done here.
            Sampling from the conditional distribution
        """
        for k in range(K):
            Xk = dataset_total[zs == k]
            samples_per_cluster[k] = Xk.shape[0]
            lambda_post = prec0 + samples_per_cluster[k]*prec
            cov_post = np.linalg.inv(lambda_post)
            left = cov_post
            right = prec0 @ start_mean + samples_per_cluster[k]*prec @ np.mean(Xk, axis=0)
            means_post = left @ right
            means[k] = multi_variate_normal.rvs(means_post, cov_post)
        """
            Below we plot - by using the corresponding indices for each cluster.
        """
    #print("Gibbs Sampling Done! - Plotting now....\n")
    km.fit(dataset_total)
    predicted = km.predict(dataset_total)
    Y=np.zeros(len(dataset_total))
    for k in range(K):
        Y[C[k]==1]=k+1
    fig, ax = plt.subplots(nrows=1, ncols=2)
    fig.set_figheight(5)
    fig.set_figwidth(15)
    plt.subplot(1,2,1)
    plt.title('KMeans++ (k='+str(k_)+')')
    plt.xlabel('X Axis')
    plt.ylabel('Y Axis')
    plt.scatter(dataset_total[:,0],dataset_total[:,1],c=predicted,cmap='plasma')
    plt.subplot(1,2,2)
    plt.title('CRP Dirichlet Process - Found '+str(K) + ' after 20 iterations')
    plt.scatter(dataset_total[:,0],dataset_total[:,1],c=Y,cmap='plasma')
    plt.xlabel('X Axis')
    plt.ylabel('Y Axis')
    plt.show()

In [7]:
#nbi:hide_in
from ipywidgets import interact
interact(chinese_restaurant_process_clustering, alpha=(0.1,10000), seed=(1,5),num_cluster=(1,8),k_=(1,10))

interactive(children=(FloatSlider(value=5000.05, description='alpha', max=10000.0, min=0.1), IntSlider(value=3…

<function __main__.chinese_restaurant_process_clustering(alpha, seed, k_, num_cluster)>

## Sammed S Kagi | Deepak Narayanan | Girish Chandar | Shivji Bhagat